<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/actividades%20extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
!pip install pyxlsb

In [3]:
#designar nomes de ficheiros
template_acordos_pagamento="012024_TEMPLATE_ACORDOS_PAGAMENTO_v3.xlsx"
template_pagamento="012024_TEMPLATE_PAGAMENTO.xlsx"

erro_template_acordos_pagamento="Erro_acordos_pagamento.xlsx"

In [4]:
#upload dos ficheiros "template acordos pagamento" e "template pagamento"
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()


Saving 012024_TEMPLATE_ACORDOS_PAGAMENTO_v3.xlsx to 012024_TEMPLATE_ACORDOS_PAGAMENTO_v3.xlsx


Saving 012024_TEMPLATE_PAGAMENTO.xlsx to 012024_TEMPLATE_PAGAMENTO.xlsx


In [ ]:
#ler a primeira folha (original) do "templaye_acordos_pagamento"

acordos_pagamento=pd.read_excel(template_acordos_pagamento, sheet_name=0)

In [ ]:
#visualização da dataframe "acordos_pagamento"
acordos_pagamento.head()

In [ ]:
#formatar a coluna "Payment Agreement Date" para retirar a hora

acordos_pagamento["Payment Agreement Date"]=acordos_pagamento.iloc[:, -1].apply(lambda i: i.split(" ")[0])

acordos_pagamento.head()

In [ ]:
#eliminar duplicados

acordos_pagamento2=acordos_pagamento.copy()

print(acordos_pagamento2.shape)

acordos_pagamento2.drop_duplicates(ignore_index=True, inplace=True)

print(acordos_pagamento2.shape)

acordos_pagamento2.head()

In [ ]:
#filtrar acordos de pagamento por "em curso" (na coluna status) e substituir "em curso" por "aprovado"

acordos_aprovado=acordos_pagamento2[acordos_pagamento2["Status"].str.contains("em curso", case=False)]

acordos_aprovado.loc[acordos_aprovado["Status"].str.contains("em curso", case=False), "Status 2"]="Aprovado"

acordos_aprovado.drop("Status", axis=1, inplace=True)

acordos_aprovado=acordos_aprovado.reindex(["Payment Agreement ID", "Service Request ID", "Status 2", "Payment Agreement Date"], axis=1)

acordos_aprovado.columns=["Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date"]

print(acordos_aprovado.shape)

acordos_aprovado.head()

In [ ]:
#fazer upload do ficheiro de erro (SRs de acordos de pagamento que não existem em RI)
from google.colab import files
uploaded3 = files.upload()


In [ ]:
#ler ficheiro erro_acordos_pagamento
erro_acordos_pagamento=pd.read_excel(erro_template_acordos_pagamento, sheet_name=0)

In [ ]:
#visualizar ficheiro erro_acordos_pagamento
erro_acordos_pagamento.iloc[:, :5].head()

In [ ]:
# formatar ficheiro erro_acordos_pagamento
erro_acordos_pagamento=erro_acordos_pagamento.iloc[2:, :5]

erro_acordos_pagamento.columns=["Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date", "RI Info"]

erro_acordos_pagamento.iloc[:, -2]=erro_acordos_pagamento.iloc[:,-2].apply(lambda i: i.strftime("%Y/%m/%d"))

erro_acordos_pagamento.index=range(erro_acordos_pagamento.iloc[:, 0].count())

print(erro_acordos_pagamento.shape)
erro_acordos_pagamento.head()

In [ ]:
#eliminar linhas na dataframe "acordos_aprovado" cujas SRs não estão em RI (cruzar com info da dataframe "erro_acordos_pagamento")

#criar colunas "chave" em "acordos_aprovado" e "erro_acordos_pagamento"

acordos_aprovado["chave"]=acordos_aprovado["Payment Agreement ID"].astype("str") + acordos_aprovado["Service Request ID"].astype("str")

erro_acordos_pagamento["chave"]=\
erro_acordos_pagamento["Payment Agreement ID"].astype("str") + erro_acordos_pagamento["Service Request ID"].astype("str")




In [ ]:
#confirmar se todas as chaves de "acordos_aprovado" estão em "erro_acordos_pagamento"
print(acordos_aprovado.loc[~acordos_aprovado["chave"].isin(erro_acordos_pagamento["chave"]), "chave"].count())

In [ ]:
#eliminar linhas na dataframe "acordos_aprovado" cujas SRs não estão em RI (cruzar com info da dataframe "erro_acordos_pagamento")


#quantas SRS não foram encontradas em RI:

print(erro_acordos_pagamento.loc[erro_acordos_pagamento.iloc[:, -2].isna()==False,"RI Info"].count())

#lista_erro inclui as linhas de SRs que não existem em RI
lista_erro=erro_acordos_pagamento.loc[erro_acordos_pagamento.iloc[:, -2].isna()==False, "chave"].tolist()

acordos_aprovado2=acordos_aprovado[~acordos_aprovado["chave"].isin(lista_erro)]

#apagar coluna "chave" em "acordos_aprovado2"

acordos_aprovado2.drop("chave", axis=1, inplace=True)

print(acordos_aprovado2.shape)

acordos_aprovado2.head()


In [ ]:
#eliminar eventuais registos duplicados nas colunas "Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date" - *TESTAR

print(acordos_aprovado2.shape)

acordos_aprovado2["quant"]=acordos_aprovado2.apply(lambda i: "".join(i.values.astype("str")), axis=1)

if acordos_aprovado2["quant"].duplicated().sum()>0:
  indice=[]
  indice=acordos_aprovado2[acordos_aprovado2["quant"].duplicated()].index
  acordos_aprovado2.drop(index=indice, inplace=True)

#eliminar coluna "quant"
acordos_aprovado2.drop("quant", axis=1, inplace=True)

#fazer ordenação por todas as colunas excepto "status"

acordos_aprovado2["Service Request ID 2"]=acordos_aprovado2.iloc[:, 1].apply(lambda i: i.split("-")[1])

acordos_aprovado2.sort_values(by=["Payment Agreement ID", "Payment Agreement Date", "Service Request ID 2"], ascending=[True, False, True], inplace=True)

acordos_aprovado2.drop("Service Request ID 2", axis=1, inplace=True)


print(acordos_aprovado2.shape)

In [ ]:
#formatar coluna com data type e ordenar tabela por "Payment Agreement ID" e "Payment Agreement Date"

acordos_aprovado2["Payment Agreement Date"]=acordos_aprovado2["Payment Agreement Date"].apply(lambda i: datetime.strptime(i, "%d/%m/%Y").date())


acordos_aprovado3=acordos_aprovado2.sort_values(["Payment Agreement ID", "Payment Agreement Date"], ascending=[True, False], ignore_index=True)

In [174]:
#criar colunas "contagem" e "índice"


def contagem (db, col):
  #criar lista para coluna "contagem"
  listaa=list(map(lambda i: db.loc[db[col]==i, col].count(), db[col].astype("str").tolist()))

  #listaa=db.groupby(col) [col].count()
  return listaa


def indice (db, col):
  indice=[]
  for i in db.groupby(col) [col].count().tolist():
    for j in range(1, i+1):
      indice.append(j)
  return indice

acordos_aprovado3["contagem"]=contagem(acordos_aprovado3, "Payment Agreement ID")

acordos_aprovado3["indice"]=indice(acordos_aprovado3, "Payment Agreement ID")




In [ ]:
#ver acordos_aprovado3

print(acordos_aprovado3.shape)


acordos_aprovado3.head()

In [ ]:
#filtrar linhas com índice igual a 1 e remover colunas "contagem" e "índice"

acordos_aprovado4=acordos_aprovado3[acordos_aprovado3["indice"]==1]
acordos_aprovado4.drop(["contagem", "indice"], axis=1, inplace=True)
print(acordos_aprovado4.shape)

acordos_aprovado4.head()

In [ ]:
#ordenar "acordos_aprovado4" por "Service Request ID" (ordem ascendente)

acordos_aprovado5=acordos_aprovado4.sort_values(by=["Service Request ID", "Payment Agreement Date"], ascending=[True, False])

acordos_aprovado5.head()

In [ ]:
#criar coluna "contagem" e "índice" na tabela "acordos_aprovado5"

acordos_aprovado5["contagem"]=contagem(acordos_aprovado5, "Service Request ID")

acordos_aprovado5["indice"]=indice(acordos_aprovado5, "Service Request ID")
print(acordos_aprovado5.shape)
acordos_aprovado5.head()

In [ ]:
#filtrar linhas com índice igual a 1

acordos_aprovado6=acordos_aprovado5[acordos_aprovado5["indice"]==1]
print(acordos_aprovado6.shape)

In [ ]:
#recolocar coluna "Payment Agreement Date" como texto
acordos_aprovado6["Payment Agreement Date"]=acordos_aprovado5["Payment Agreement Date"].apply(lambda i: i.strftime("%d/%m/%Y"))

acordos_aprovado6.head()

In [ ]:
#exportar tabela "acordos_aprovado5" e fazer download

acordos_aprovado6.to_excel("acordos de pagamento.xlsx", index=False)

files.download("acordos de pagamento.xlsx")

In [92]:
#ler e ver o ficheiro template_pagamento
template_pagamentodb=pd.read_excel(template_pagamento, sheet_name=0)

template_pagamentodb.head()

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4
0,NaN,1-44707521725,7,2024-02-07,0
1,NaN,1-44751135164,0,2024-01-09,0
2,963273981.0,1-44752802128,0,2024-01-31,Service Request Id does not exist;\n
3,848853839.0,1-44746957765,1,2024-01-31,0
4,NaN,1-44736760761,0,2024-01-31,0


In [93]:
#substituir NaN por "NULL" na 1ª coluna

template_pagamentodb.iloc[:, 0]=template_pagamentodb.iloc[:, 0].fillna("NULL")

#ver quantos NaN existem nas colunas
print(template_pagamentodb.isna().sum())

template_pagamentodb.head()

Billing Account         0
Service Request ID      0
Payment Days            0
Invoice Payment Date    0
Unnamed: 4              0
dtype: int64


,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4
0,NULL,1-44707521725,7,2024-02-07,0
1,NULL,1-44751135164,0,2024-01-09,0
2,963273981.0,1-44752802128,0,2024-01-31,Service Request Id does not exist;\n
3,848853839.0,1-44746957765,1,2024-01-31,0
4,NULL,1-44736760761,0,2024-01-31,0


In [94]:
#eliminar linhas duplicadas no template_pagamentodb
print(template_pagamentodb.shape)
print(f"o nº de duplicados é {template_pagamentodb.duplicated().sum()}")
template_pagamentodb.drop_duplicates(ignore_index=True, inplace=True)
print(template_pagamentodb.shape)

(6174, 5)
o nº de duplicados é 12
(6162, 5)


In [120]:
template_pagamentodb[template_pagamentodb["Billing Account"]==960779142]

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave
314,960779142.0,1-44639345257,0,2024-01-19,0,9607791421-4463934525702024-01-190
347,960779142.0,1-44674045289,0,2024-01-19,Service Request Id does not exist;\n,9607791421-4467404528902024-01-19Service Reque...


In [95]:
#ler o ficheiro que RI devolve como erro no upload do ficheiro template_pagamentodb - actualizar nome da folha se necessário

erro_pagamento=pd.read_excel(template_pagamento, sheet_name="ERRO")

erro_pagamento.head()

,RAID Incentives,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Invoice Payments,NaN,NaN,NaN,NaN
1,Billing Account,Service Request Id,Payment Days,Invoice Payment Date,NaN
2,NaN,1-44707521725,7,2024-02-07 00:00:00,NaN
3,NaN,1-44751135164,0,2024-01-09 00:00:00,NaN
4,963273981,1-44752802128,0,2024-01-31 00:00:00,Service Request Id does not exist;\n


In [96]:
#formatar folha "erro pagamento"
erro_pagamento2=erro_pagamento.iloc[2:, :5]

erro_pagamento2.columns=erro_pagamento.iloc[1, :5]

#separar data da hora
erro_pagamento2["Invoice Payment Date"]=pd.to_datetime(erro_pagamento2["Invoice Payment Date"])

print(erro_pagamento2.shape)
erro_pagamento2.head()


(6174, 5)


1,Billing Account,Service Request Id,Payment Days,Invoice Payment Date,NaN
2,NaN,1-44707521725,7,2024-02-07,NaN
3,NaN,1-44751135164,0,2024-01-09,NaN
4,963273981,1-44752802128,0,2024-01-31,Service Request Id does not exist;\n
5,848853839,1-44746957765,1,2024-01-31,NaN
6,NaN,1-44736760761,0,2024-01-31,NaN


In [97]:
#substituir NaN na 1ª coluna de erro_pagamento2

erro_pagamento2.iloc[:, 0]=erro_pagamento2.iloc[:, 0].fillna("NULL")

#substituir NaN na 5ª coluna onde se sinalizam os erros

erro_pagamento2.iloc[:, 4]=erro_pagamento2.iloc[:, 4].fillna("")

print(erro_pagamento2.isna().sum())

erro_pagamento2.head()



1
Billing Account         0
Service Request Id      0
Payment Days            0
Invoice Payment Date    0
NaN                     0
dtype: int64


1,Billing Account,Service Request Id,Payment Days,Invoice Payment Date,NaN
2,NULL,1-44707521725,7,2024-02-07,
3,NULL,1-44751135164,0,2024-01-09,
4,963273981,1-44752802128,0,2024-01-31,Service Request Id does not exist;\n
5,848853839,1-44746957765,1,2024-01-31,
6,NULL,1-44736760761,0,2024-01-31,


In [98]:
#criar coluna "chave" em template_pagamentodb e erro_pagamento2

def func_chave(db, col="Billing Account"):
  db=db.astype("str")
  db[col]=db[col].apply(lambda i: i.split(".")[0])
  db["chave"]=db.apply(lambda i: "".join(i.values.astype("str")).strip(), axis=1)
  return db["chave"]

template_pagamentodb["chave"]=func_chave(template_pagamentodb)

erro_pagamento2["chave"]=func_chave(erro_pagamento2)



In [99]:
#excluir todas as linhas que devolvem erro por SR não existente


template_pagamentodb2=template_pagamentodb.loc[~template_pagamentodb["chave"].isin(erro_pagamento2["chave"].tolist()), :]

print(template_pagamentodb2.shape)

template_pagamentodb2.head()

(6068, 6)


,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave
0,NULL,1-44707521725,7,2024-02-07,0,NULL1-4470752172572024-02-070
1,NULL,1-44751135164,0,2024-01-09,0,NULL1-4475113516402024-01-090
3,848853839.0,1-44746957765,1,2024-01-31,0,8488538391-4474695776512024-01-310
4,NULL,1-44736760761,0,2024-01-31,0,NULL1-4473676076102024-01-310
5,839782114.0,1-44745529027,1,2024-01-31,0,8397821141-4474552902712024-01-310


In [121]:
template_pagamentodb2[template_pagamentodb2["Billing Account"]==960779142]

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave
314,960779142.0,1-44639345257,0,2024-01-19,0,9607791421-4463934525702024-01-190


In [100]:
#eliminar duplicados na tabela "template_pagamentodb2" e ordenar tabela

print(f"existem {template_pagamentodb2.duplicated().sum()} linhas duplicadas")
template_pagamentodb3=template_pagamentodb2.drop_duplicates()


existem 0 linhas duplicadas


In [101]:
#ordenar tabela por "billing account" e payment days (ascendente) e criar coluna "contagem" e "índice"

template_pagamentodb4=template_pagamentodb3.sort_values(by=["Billing Account", "Payment Days"])

template_pagamentodb4["contagem"]=contagem(template_pagamentodb4, "Billing Account")
template_pagamentodb4["indice"]=indice(template_pagamentodb4, "Billing Account")

template_pagamentodb4.head()

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave,contagem,indice
421,240752401.0,1-44635495767,0,2024-01-19,0,2407524011-4463549576702024-01-190,1,1
563,434857301.0,1-44650294908,0,2024-01-17,0,4348573011-4465029490802024-01-170,1,1
1194,441543201.0,1-44555998069,0,2024-01-03,0,4415432011-4455599806902024-01-030,1,1
1043,643720801.0,1-44573729588,0,2024-01-08,0,6437208011-4457372958802024-01-080,1,1
1008,823282775.0,1-44570623973,4,2024-01-08,0,8232827751-4457062397342024-01-080,1,1


In [122]:
template_pagamentodb4[template_pagamentodb4["Billing Account"]==960779142]

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave,contagem,indice
314,960779142.0,1-44639345257,0,2024-01-19,0,9607791421-4463934525702024-01-190,1,1


In [102]:
#levar billing accounts diferentes de null com indice 1 e todas as billing accounts null

mask_null1=template_pagamentodb4.iloc[:, 0]=="NULL"
maskindice1=(template_pagamentodb4["indice"]==1) & (template_pagamentodb4.iloc[:, 0]!="NULL")

template_pagamentodb5=pd.concat([template_pagamentodb4.loc[mask_null1], template_pagamentodb4.loc[maskindice1]], ignore_index=True)

print(template_pagamentodb5.shape)
template_pagamentodb5.head()

(6039, 8)


,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave,contagem,indice
0,NULL,1-44751135164,0,2024-01-09,0,NULL1-4475113516402024-01-090,5827,1
1,NULL,1-44736760761,0,2024-01-31,0,NULL1-4473676076102024-01-310,5827,2
2,NULL,1-44730882269,0,2024-01-31,0,NULL1-4473088226902024-01-310,5827,3
3,NULL,1-44730865641,0,2024-01-31,0,NULL1-4473086564102024-01-310,5827,4
4,NULL,1-44697188442,0,2024-01-15,0,NULL1-4469718844202024-01-150,5827,5


In [103]:
#apagar colunas antigas "contagem" e "índice"

template_pagamentodb5.drop(["contagem", "indice"], axis=1, inplace=True)

template_pagamentodb5.head()


,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave
0,NULL,1-44751135164,0,2024-01-09,0,NULL1-4475113516402024-01-090
1,NULL,1-44736760761,0,2024-01-31,0,NULL1-4473676076102024-01-310
2,NULL,1-44730882269,0,2024-01-31,0,NULL1-4473088226902024-01-310
3,NULL,1-44730865641,0,2024-01-31,0,NULL1-4473086564102024-01-310
4,NULL,1-44697188442,0,2024-01-15,0,NULL1-4469718844202024-01-150


In [123]:
template_pagamentodb5[template_pagamentodb5["Billing Account"]==960779142]

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave
5927,960779142.0,1-44639345257,0,2024-01-19,0,9607791421-4463934525702024-01-190


In [162]:
#ordenar tabela "template_pagamentodb5" por "service request id" e "payment days"

template_pagamentodb6=template_pagamentodb5.sort_values(by=["Service Request ID", "Payment Days"])

print(template_pagamentodb6.shape)

(6039, 6)


In [164]:
template_pagamentodb6.columns

Index(['Billing Account', 'Service Request ID', 'Payment Days',
       'Invoice Payment Date', 'Unnamed: 4', 'chave'],
      dtype='object')

In [170]:
#for i, j in zip(template_pagamentodb6["Service Request ID"],template_pagamentodb6.groupby("Service Request ID")["Service Request ID"].count().tolist()):
  #if i==960779142:
    #print(j)

pd.DataFrame(template_pagamentodb6.groupby("Service Request ID")["Service Request ID"].count(), columns=["sr", "quant"])

df=pd.DataFrame(template_pagamentodb6.groupby("Service Request ID")["Service Request ID"].count())

df.loc[df.iloc[:, 0]=="1-44639345257", :]


,Service Request ID
Service Request ID,


In [171]:
#criar colunas "contagem" e "índice" - por SR

template_pagamentodb6["contagem"]=contagem(template_pagamentodb6, "Service Request ID")
template_pagamentodb6["indice"]=indice(template_pagamentodb6, "Service Request ID")

template_pagamentodb6.head()

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave,contagem,indice
1040,NULL,1-43864551544,4001,2024-01-17,0,NULL1-4386455154440012024-01-170,NaN,1
1041,NULL,1-43905949875,4001,2024-01-05,0,NULL1-4390594987540012024-01-050,NaN,1
1042,NULL,1-43930671446,4001,2024-01-15,0,NULL1-4393067144640012024-01-150,NaN,1
1043,NULL,1-43948307301,4001,2024-01-05,0,NULL1-4394830730140012024-01-050,NaN,1
1044,NULL,1-43983837488,4001,2024-01-29,0,NULL1-4398383748840012024-01-290,NaN,1


In [173]:
#template_pagamentodb6[template_pagamentodb6["Billing Account"].astype("str")=="960779142.0"]

#template_pagamentodb6.dtypes

template_pagamentodb6[template_pagamentodb6["Service Request ID"]=="1-44639345257"]

,Billing Account,Service Request ID,Payment Days,Invoice Payment Date,Unnamed: 4,chave,contagem,indice
107,NULL,1-44639345257,0,2024-01-19,0,NULL1-4463934525702024-01-190,NaN,1
5927,960779142.0,1-44639345257,0,2024-01-19,0,9607791421-4463934525702024-01-190,NaN,2


In [133]:
lista=list(map(lambda i: template_pagamentodb6.loc[template_pagamentodb6["Billing Account"]==i, "Billing Account"].count(), \
         template_pagamentodb6["Billing Account"].tolist()))

for i, j in zip(template_pagamentodb6["Billing Account"], lista):
  if str(i)=="960779142.0":
    print(j)


1


In [106]:
#filtrar srs com payment days entre 0 e 7 e iguais a 4001

def filtro_sr (db, col_days="Payment Days", col_contagem="contagem", col_sr="Service Request ID"):
  #filtrar srs repetidas
  mask_null2=db[col_contagem]>1

  #filtrar por contagem >1 e fazer somatório da coluna "payment days" agrupando por SR
  db1=db[mask_null2].groupby(col_sr) [col_days].sum().reset_index()

  #identificar srs com somatório inferior a 4001
  valores=db1[db1.iloc[:, 1]<4001].iloc[:, 0].values

  template_pagamento_final=pd.concat([db.loc[(~db[col_sr].isin(valores)) & (db["contagem"]>1), :], db.loc[db[col_contagem]==1, :]], ignore_index=True)

  #db.drop()
  #mask_days=(db[col_days].between(0, 7, "both")) | (db[col_days]>4000)
  #db2=db[(mask_days) & (mask_null2)] #db2 com coluna contagem >1
  #db1=db[~mask_null2] #db1 com coluna=1

  #template_pagamento_final=pd.concat([db1, db2])#juntar as 2 tabelas

  template_pagamento_final.sort_values(by=[col_sr, col_days], inplace=True, ignore_index=True)
  return template_pagamento_final

template_pagamentodb7=filtro_sr(template_pagamentodb6)

In [112]:
#template_pagamentodb6.loc[template_pagamentodb6["contagem"]>1, "contagem"].count()

#template_pagamentodb6[template_pagamentodb6["contagem"]>1].groupby("Service Request ID") ["Payment Days"].sum()

#template_pagamentodb7.loc[template_pagamentodb7["Service Request ID"]=="1-44639345257", :]

#template_pagamentodb7.loc[template_pagamentodb7["contagem"]==1, :]

#template_pagamentodb7.iloc[:, 0].astype("int")

template_pagamentodb7["Billing Account"]=template_pagamentodb7.loc[template_pagamentodb7.iloc[:, 0]!="NULL", "Billing Account"].astype("int")

In [116]:
template_pagamentodb7.to_excel("pagamento facturas.xlsx", index=False)

files.download("pagamento facturas.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>